In [ ]:
import os
import cohere
import openai
import requests
from config import config
import yfinance as yf

COHERE_API_KEY = config["COHERE_API_KEY"]
PERPLEXITY_API_KEY = config["PERPLEXITY_API_KEY"]
FINANCIAL_MODELING_PREP_API_KEY = config["FINANCIAL_MODELING_PREP_API_KEY"]
SAMBA_API_KEY = config["SAMBA_API"]
OPENAI_API_KEY = config["OPENAI_API_KEY"]

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


co = cohere.Client(COHERE_API_KEY)
perclient = openai.OpenAI(api_key=PERPLEXITY_API_KEY, base_url="https://api.perplexity.ai")
sambaclient = openai.OpenAI(api_key=SAMBA_API_KEY, base_url="https://api.sambanova.ai/v1")
openclient = openai.OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
samba_response = sambaclient.chat.completions.create(
        model="Meta-Llama-3.1-70B-Instruct",
        messages=[
            {
                "role": "system",
                "content": (
                    "You are an intelligent algorithmic trader who can explain the ups and downs of each algorithm when asked about it."
                ),
            },
            {
                "role": "user",
                "content": "Explain the pros and cons of this strategy while comparing it to the buy-and-hold strategy. Explain how it works and why it is effective. " + "20 day short term moving average and 50 day long term moving average",
            },
        ]
    )

samba_response.choices[0].message.content

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The strategy you\'re referring to is known as the Moving Average Crossover (MAC) strategy, which uses two moving averages (MAs) with different time periods to generate buy and sell signals. In this case, we\'re using a 20-day short-term MA and a 50-day long-term MA.\n\n**How it works:**\n\n1. Calculate the 20-day MA and 50-day MA of the stock\'s price.\n2. When the 20-day MA crosses above the 50-day MA, it\'s a bullish signal, indicating a potential uptrend. This is known as a "golden cross."\n3. When the 20-day MA crosses below the 50-day MA, it\'s a bearish signal, indicating a potential downtrend. This is known as a "death cross."\n4. Buy the stock when the golden cross occurs and sell when the death cross occurs.\n\n**Pros:**\n\n1. **Trend following**: The MAC strategy helps you ride the trend, as the moving averages will follow the price movement.\n2. **Risk management**: The strategy provi

In [3]:
def get_stock_price(symbol):
    """
    Fetch the current stock price for the given symbol, the current volume, the average price 50d and 200d, EPS, PE and the next earnings Announcement.
    """
    url = f"https://financialmodelingprep.com/api/v3/quote-order/{symbol}?apikey={FINANCIAL_MODELING_PREP_API_KEY}"
    response = requests.get(url)
    data = response.json()
    try:
        price = data[0]['price']
        volume = data[0]['volume']
        priceAvg50 = data[0]['priceAvg50']
        priceAvg200 = data[0]['priceAvg200']
        eps = data[0]['eps']
        pe = data[0]['pe']
        earningsAnnouncement = data[0]['earningsAnnouncement']
        return {"symbol": symbol.upper(), "price": price, "volume":volume,"priceAvg50":priceAvg50, "priceAvg200":priceAvg200, "EPS":eps, "PE":pe, "earningsAnnouncement":earningsAnnouncement }
    except (IndexError, KeyError):
        return {"error": f"Could not fetch price for symbol: {symbol}"}

def get_company_financials(symbol):
    """
    Fetch basic financial information for the given company symbol such as the industry, the sector, the name of the company, and the market capitalization.
    """
    url = f"https://financialmodelingprep.com/api/v3/profile/{symbol}?apikey={FINANCIAL_MODELING_PREP_API_KEY}"
    response = requests.get(url)
    data = response.json()
    try:
        results = data[0]
        financials = {
            "symbol": results["symbol"],
            "companyName": results["companyName"],
            "marketCap": results["mktCap"],
            "industry": results["industry"],
            "sector": results["sector"],
            "website": results["website"],
            "beta":results["beta"],
            "price":results["price"],
        }
        return financials
    except (IndexError, KeyError):
        return {"error": f"Could not fetch financials for symbol: {symbol}"}

def get_income_statement(symbol):
    """
    Fetch last income statement for the given company symbol such as revenue, gross profit, net income, EBITDA, EPS.
    """
    url = f"https://financialmodelingprep.com/api/v3/income-statement/{symbol}?period=annual&apikey={FINANCIAL_MODELING_PREP_API_KEY}"
    response = requests.get(url)
    data = response.json()
    try:
        results = data[0]
        financials = {
            "date": results["date"],
            "revenue": results["revenue"],
            "gross profit": results["grossProfit"],
            "net Income": results["netIncome"],
            "ebitda": results["ebitda"],
            "EPS": results["eps"],
            "EPS diluted":results["epsdiluted"]
        }
        return data, financials
    except (IndexError, KeyError):
        return {"error": f"Could not fetch financials for symbol: {symbol}"}

In [4]:
from swarm import Agent

manager_agent = Agent(
    name="Manager Agent",
    instructions="Determine which agent is best suited to handle the user's request, and transfer the conversation to that agent.",
)
stock_price_agent = Agent(
    name="Stock Price Agent",
    instructions="Fetch Hsitorical prices for a given stock symbol, the current volume, the average price 50d and 200d, EPS, PE and the next earnings Announcement.",
    functions=[get_stock_price],
)

company_basic_information_agent = Agent(
    name="Company basic information Agent",
    instructions="Fetch basic financial information for the given company symbol such as the industry, the sector, the name of the company, and the market capitalization.",
    functions=[get_company_financials],
)

income_statement_agent = Agent(
    name="Income Statement Agent",
    instructions=" Fetch last income statement for the given company symbol such as revenue, gross profit, net income, EBITDA, EPS.",
    functions=[get_income_statement],
)


def transfer_back_to_manager():
    """Call this function if a user is asking about a topic that is not handled by the current agent."""
    return manager_agent


def transfer_to_stock_price():
    return stock_price_agent

def transfer_to_company_basic_info():
    return company_basic_information_agent

def transfer_to_income_statement():
    return income_statement_agent


manager_agent.functions = [transfer_to_stock_price, transfer_to_company_basic_info, transfer_to_income_statement]
stock_price_agent.functions.append(transfer_back_to_manager)
company_basic_information_agent.functions.append(transfer_back_to_manager)
income_statement_agent.functions.append(transfer_back_to_manager)

In [5]:
# You can get this code from "from swarm.repl import run_demo_loop"==>I've extracted here to add "exit" to break the loop.
import json

from swarm import Swarm

def process_and_print_streaming_response(response):
    content = ""
    last_sender = ""

    for chunk in response:
        if "sender" in chunk:
            last_sender = chunk["sender"]

        if "content" in chunk and chunk["content"] is not None:
            if not content and last_sender:
                print(f"\033[94m{last_sender}:\033[0m", end=" ", flush=True)
                last_sender = ""
            print(chunk["content"], end="", flush=True)
            content += chunk["content"]

        if "tool_calls" in chunk and chunk["tool_calls"] is not None:
            for tool_call in chunk["tool_calls"]:
                f = tool_call["function"]
                name = f["name"]
                if not name:
                    continue
                print(f"\033[94m{last_sender}: \033[95m{name}\033[0m()")

        if "delim" in chunk and chunk["delim"] == "end" and content:
            print()  # End of response message
            content = ""

        if "response" in chunk:
            return chunk["response"]


def pretty_print_messages(messages) -> None:
    for message in messages:
        if message["role"] != "assistant":
            continue

        # print agent name in blue
        print(f"\033[94m{message['sender']}\033[0m:", end=" ")

        # print response, if any
        if message["content"]:
            print(message["content"])

        # print tool calls in purple, if any
        tool_calls = message.get("tool_calls") or []
        if len(tool_calls) > 1:
            print()
        for tool_call in tool_calls:
            f = tool_call["function"]
            name, args = f["name"], f["arguments"]
            arg_str = json.dumps(json.loads(args)).replace(":", "=")
            print(f"\033[95m{name}\033[0m({arg_str[1:-1]})")


def run_demo_loop(
    starting_agent, context_variables=None, stream=False, debug=False
) -> None:
    client = Swarm()
    print("Starting Swarm CLI 🐝")

    messages = []
    agent = starting_agent

    while True:
        user_input = input("\033[90mUser\033[0m: ")
        if user_input.lower() == "exit":
            break
        messages.append({"role": "user", "content": user_input})

        response = client.run(
            agent=agent,
            messages=messages,
            context_variables=context_variables or {},
            stream=stream,
            debug=debug,
        )

        if stream:
            response = process_and_print_streaming_response(response)
        else:
            pretty_print_messages(response.messages)

        messages.extend(response.messages)
        agent = response.agent

In [ ]:
run_demo_loop(manager_agent, debug=False)

Starting Swarm CLI 🐝
Manager Agent: 
transfer_to_income_statement()
transfer_to_income_statement()
Income Statement Agent: 
get_income_statement("symbol"= "AAPL")
get_income_statement("symbol"= "005930.KS")
Income Statement Agent: I was able to fetch the latest revenue data for Apple but encountered an issue when trying to fetch the revenue for Samsung.

### Apple Inc. (AAPL)
- **Date:** September 28, 2024
- **Revenue:** $391.035 billion
- **Gross Profit:** $180.683 billion
- **Net Income:** $93.736 billion
- **EBITDA:** $134.661 billion
- **EPS (Earnings Per Share):** $6.11

Unfortunately, I couldn't retrieve the revenue information for Samsung due to a data fetching issue. If you have any other requests or need more details, feel free to ask!
Income Statement Agent: 
get_income_statement("symbol"= "AAPL")
get_income_statement("symbol"= "NVDA")
Income Statement Agent: Here's the comparison of the latest financial results for Apple and Nvidia:

### Apple Inc. (AAPL)
- **Date:** Septemb